In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

In [3]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [4]:
from sklearn.preprocessing import *

In [5]:
dataset = pd.read_csv("/content/weatherHistory.csv")

FileNotFoundError: ignored

In [ ]:
dataset.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [ ]:
dataset.isnull().sum() #there are 517 null values which should be replaced

Formatted Date                0
Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Loud Cover                    0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64

In [ ]:
dataset["Precip Type"] = dataset["Precip Type"].fillna(dataset["Precip Type"].mode()[0]) #Fill null values with mode value - Rain

In [ ]:
dataset.isna().sum() 

Formatted Date              0
Summary                     0
Precip Type                 0
Temperature (C)             0
Apparent Temperature (C)    0
Humidity                    0
Wind Speed (km/h)           0
Wind Bearing (degrees)      0
Visibility (km)             0
Loud Cover                  0
Pressure (millibars)        0
Daily Summary               0
dtype: int64

In [ ]:
dataset["Formatted Date"] = pd.to_datetime(dataset["Formatted Date"], format = "%Y-%m-%d %H:%M:%S.%f %z") 

In [ ]:
#"loud cover" has only one unique value
dataset = dataset.drop(["Loud Cover","Daily Summary"], axis=1) 

In [ ]:
dataset.corr() 

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars)
Temperature (C),1.000000,0.992629,-0.632255,0.008957,0.029988,0.392847,-0.005447
Apparent Temperature (C),0.992629,1.000000,-0.602571,-0.056650,0.029031,0.381718,-0.000219
Humidity,-0.632255,-0.602571,1.000000,-0.224951,0.000735,-0.369173,0.005454
Wind Speed (km/h),0.008957,-0.056650,-0.224951,1.000000,0.103822,0.100749,-0.049263
Wind Bearing (degrees),0.029988,0.029031,0.000735,0.103822,1.000000,0.047594,-0.011651
Visibility (km),0.392847,0.381718,-0.369173,0.100749,0.047594,1.000000,0.059818
Pressure (millibars),-0.005447,-0.000219,0.005454,-0.049263,-0.011651,0.059818,1.000000


In [ ]:
#apparent temperature is highly correlated to temperature and should be removed
dataset = dataset.drop(["Apparent Temperature (C)"], axis=1) 

In [ ]:
X = dataset

In [ ]:
dataset["year"] = dataset["Formatted Date"].apply(lambda x: x.year)
dataset["month"] = dataset["Formatted Date"].apply(lambda x: x.month)
dataset["day"] = dataset["Formatted Date"].apply(lambda x: x.day)

In [ ]:
dataset = dataset.drop(["Formatted Date"], axis=1)

In [ ]:
#Label Encoding Text Values
le = LabelEncoder()
le.fit(dataset["Summary"])
dataset["Summary"] = le.transform(dataset["Summary"])
le.fit(dataset["Precip Type"])
dataset["Precip Type"] = le.transform(dataset["Precip Type"])

In [ ]:
y = dataset["Temperature (C)"]
X = dataset.drop(["Temperature (C)"], axis = 1)

In [ ]:
X

,Summary,Precip Type,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),year,month,day
0,19,0,0.89,14.1197,251.0,15.8263,1015.13,2006,4,1
1,19,0,0.86,14.2646,259.0,15.8263,1015.63,2006,4,1
2,17,0,0.89,3.9284,204.0,14.9569,1015.94,2006,4,1
3,19,0,0.83,14.1036,269.0,15.8263,1016.41,2006,4,1
4,17,0,0.83,11.0446,259.0,15.8263,1016.51,2006,4,1
...,...,...,...,...,...,...,...,...,...,...
96448,19,0,0.43,10.9963,31.0,16.1000,1014.36,2016,9,9
96449,19,0,0.48,10.0947,20.0,15.5526,1015.16,2016,9,9
96450,19,0,0.56,8.9838,30.0,16.1000,1015.66,2016,9,9
96451,19,0,0.60,10.5294,20.0,16.1000,1015.95,2016,9,9


In [ ]:
#Scale Input values
sc = StandardScaler()
sc.fit(X)
X = pd.DataFrame(sc.transform(X), columns=X.columns)
X

,Summary,Precip Type,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),year,month,day
0,0.686460,-0.353461,0.793470,0.478635,0.591256,1.306976,0.101685,-1.581343,-0.731859,-1.673130
1,0.686460,-0.353461,0.639996,0.499594,0.665756,1.306976,0.105960,-1.581343,-0.731859,-1.673130
2,0.227899,-0.353461,0.793470,-0.995473,0.153570,1.099586,0.108610,-1.581343,-0.731859,-1.673130
3,0.686460,-0.353461,0.486521,0.476306,0.758881,1.306976,0.112628,-1.581343,-0.731859,-1.673130
4,0.227899,-0.353461,0.486521,0.033841,0.665756,1.306976,0.113483,-1.581343,-0.731859,-1.673130
...,...,...,...,...,...,...,...,...,...,...
96448,0.686460,-0.353461,-1.559811,0.026855,-1.457488,1.372265,0.095102,1.581087,0.718056,-0.764257
96449,0.686460,-0.353461,-1.304020,-0.103556,-1.559925,1.241686,0.101942,1.581087,0.718056,-0.764257
96450,0.686460,-0.353461,-0.894753,-0.264241,-1.466800,1.372265,0.106216,1.581087,0.718056,-0.764257
96451,0.686460,-0.353461,-0.690120,-0.040680,-1.559925,1.372265,0.108696,1.581087,0.718056,-0.764257


In [ ]:
#test and train data set split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [ ]:
gbr=GradientBoostingRegressor()
rfr=RandomForestRegressor()
knr=KNeighborsRegressor()
dtr=DecisionTreeRegressor()
svr=SVR()

In [ ]:
vr=VotingRegressor([('gbr',gbr),('rfr',rfr),('knr',knr),('dtr',dtr),('svr',svr)])

In [ ]:
vr.fit(X_train,y_train)

VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                   

In [ ]:
y_pred = vr.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9437649703527355

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,y_pred))

2.2698975086424937

In [ ]:
gbr.fit(X_train,y_train)
rfr.fit(X_train,y_train)
knr.fit(X_train,y_train)
dtr.fit(X_train,y_train)
svr.fit(X_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
print(r2_score(y_test, gbr.predict(X_test)))
print(r2_score(y_test, rfr.predict(X_test)))
print(r2_score(y_test, knr.predict(X_test)))
print(r2_score(y_test, dtr.predict(X_test)))
print(r2_score(y_test, svr.predict(X_test)))
print(r2_score(y_test, y_pred))

0.8952616313759654
0.9621980877530399
0.911922431360084
0.9182673761119248
0.8846029667834658
0.9437649703527355


In [ ]:
print(np.sqrt(mean_squared_error(y_test, gbr.predict(X_test))))
print(np.sqrt(mean_squared_error(y_test, rfr.predict(X_test))))
print(np.sqrt(mean_squared_error(y_test, knr.predict(X_test))))
print(np.sqrt(mean_squared_error(y_test, dtr.predict(X_test))))
print(np.sqrt(mean_squared_error(y_test, svr.predict(X_test))))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

3.0978164851705174
1.861056974180487
2.8407658777598903
2.736531834282496
3.251622392872451
2.2698975086424937
